# Apriori learning
This code uses the python apyori module https://github.com/ymoch/apyori/blob/master/apyori.py
to analyze an online retail dataset https://www.kaggle.com/puneetbhaya/online-retail
and execute the apriori learning algorithm to extract sets of items that are frequently purchased together. 

In [ ]:
# Importing the libraries
import numpy as np
import pandas as pd

######## Data Preprocessing
dataset = pd.read_excel('Online Retail.xlsx')

#Replace null descriptions with the stock code 
for i, d in dataset[dataset['Description'].isnull()].iterrows():
    dataset['Description'][i] = "Code-" + str(d['StockCode'])
#group into baskets
grouped = dataset.groupby('InvoiceNo')

#rearrange into a list, as required by apyori
transactions = []
for name,group in grouped:
    transactions.append(list(group['Description'].map(str)))

np.random.seed(42)
#here I am sampling the contents of large baskets, the algorithm runs really long if not. 
tsamples = []
for row in transactions:
    if len(row) >= 150:
        newrow = np.random.choice(row,size=150).tolist()
    else:
        newrow = row
    tsamples.append(newrow)

### Training

In [2]:
##### Training Apriori on the dataset
from apyori import apriori
rules = apriori(tsamples, min_support = 0.005, min_confidence = 0.2, min_lift = 3, min_length = 2)

# Getting the results
results = list(rules)


In [4]:
#results print out courtesy of:
#https://www.kaggle.com/asamir/online-retail-analyze-with-association-rules
final_results = pd.DataFrame(np.random.randint(low=0, high=1, size=(len(results), 6)), columns=['GeneralRules', 'LeftRules', 'RightRules', 'Support', 'Confidence', 'Lift'])
index = 0
for g, s, i in results:
    final_results.iloc[index] = [' _&&_ '.join(list(g)), ' _&&_ '.join(list(i[0][0])), ' _&&_ '.join(list(i[0][1])), s, i[0][2], i[0][3]]
    index = index+1
# The most significant rules
final_results = final_results.sort_values('Lift', ascending=0)
#final_results.head(20)

count=1
for i, d in final_results.head(20).iterrows():
    print('Rule #'+str(count)+':')
    print(d['LeftRules'])
    print('=> '+d['RightRules'])
    print('Support: '+str(d['Support'])+' - Confidence: '+str(d['Confidence'])+' - Lift: '+str(d['Lift']))
    print('--------------------')
    count=count+1

Rule #1:
HERB MARKER BASIL _&&_ HERB MARKER CHIVES  _&&_ HERB MARKER PARSLEY _&&_ HERB MARKER ROSEMARY _&&_ HERB MARKER MINT
=> HERB MARKER THYME
Support: 0.005096525096525096 - Confidence: 0.9850746268656715 - Lift: 124.45576993083363
--------------------
Rule #2:
HERB MARKER CHIVES  _&&_ HERB MARKER ROSEMARY _&&_ HERB MARKER PARSLEY _&&_ HERB MARKER BASIL
=> HERB MARKER THYME
Support: 0.005250965250965251 - Confidence: 0.9784172661870505 - Lift: 123.61466924021761
--------------------
Rule #3:
HERB MARKER MINT _&&_ HERB MARKER ROSEMARY _&&_ HERB MARKER PARSLEY _&&_ HERB MARKER BASIL
=> HERB MARKER THYME
Support: 0.005598455598455598 - Confidence: 0.9666666666666667 - Lift: 122.13008130081302
--------------------
Rule #4:
HERB MARKER CHIVES  _&&_ HERB MARKER ROSEMARY _&&_ HERB MARKER PARSLEY _&&_ HERB MARKER MINT
=> HERB MARKER THYME
Support: 0.005482625482625483 - Confidence: 0.9659863945578231 - Lift: 122.04413472706156
--------------------
Rule #5:
HERB MARKER MINT _&&_ HERB MARKER